In [9]:
import time
import sys

In [10]:
def minimum_remaining_value(state_domain_dict, neighbor_state_dict, visited_node, key):
    
    min_constraint_key = ""
    min_constraint_len =  sys.maxsize
    for state, domain in state_domain_dict.items():
        if state in visited_node:
            continue
        if min_constraint_len > len(domain):
            min_constraint_key = state
            min_constraint_len = len(domain)
        elif min_constraint_len == len(domain):
            list1 = []
            list2 = []
            for i in neighbor_state_dict[min_constraint_key]:
                if i not in visited_node:
                    list1.append(i)
            for i in neighbor_state_dict[state]:
                if i not in visited_node:
                    list2.append(i)
            if len(list2) > len(list1):
                min_constraint_key = state
    return min_constraint_key

In [12]:
def lcv(key_state, state_domain, neighbour_list, selected_dict):
    
    num_neighbour_colors = 0
    color_dict = dict()
    for color in state_domain[key_state]:
        for neighbour in neighbour_list[key_state]:
            if neighbour not in selected_dict.keys():
                if color in state_domain[neighbour]:
                    if len(state_domain[neighbour]) - 1 == 0:
                        color_dict[color] = float("inf")
                    else:
                        num_neighbour_colors += len(state_domain[neighbour]) - 1

        if color not in color_dict.keys():
            color_dict[color] = num_neighbour_colors

    return min(color_dict, key=lambda k: color_dict[k])

In [ ]:
def degree_heuristic(state_domain_dict, neighbor_state_dict, visited_node, key):
    
    min_constraint_key = ""
    min_constraint_len =  sys.maxsize
    for state, domain in state_domain_dict.items():
        if state in visited_node:
            continue
        if min_constraint_len > len(domain):
            min_constraint_key = state
            min_constraint_len = len(domain)
        elif min_constraint_len == len(domain):
            list1 = []
            list2 = []
            for i in neighbor_state_dict[min_constraint_key]:
                if i not in visited_node:
                    list1.append(i)
            for i in neighbor_state_dict[state]:
                if i not in visited_node:
                    list2.append(i)
            if len(list2) > len(list1):
                min_constraint_key = state
    return min_constraint_key

In [13]:
selected_root_domain_values = list()
state_list = []
failed = False
back_track_count = 0
out = ""

global neighbor_state_dict, state_domain_dict
global visited_node
visited_node = set()
global selected_dict
selected_dict = dict()
global selected_backtrack_node
selected_backtrack_node = set()

def forward_checking_singleton(failed_index, state_domain_dict, neighbor_state_dict, is_backtrack=False, parent_domain_to_be_chosen=None, heuristic_value=0):
    global out
    global back_track_count
    global failed
    key = state_list[0]
    i = failed_index
    while i < len(state_list):
        previous_selected_parent_domain = None
        key = state_list[i]
        
        if heuristic_value == 1:
            key = minimum_remaining_value(state_domain_dict, neighbor_state_dict, visited_node, state_list[i])
            
        singleton_domain_list = singleton(state_domain_dict, neighbor_state_dict, key, visited_node)

        if singleton_domain_list is not []:
            for singleton_domain in singleton_domain_list:
                if singleton_domain in state_domain_dict[key]:
                    state_domain_dict[key].remove(singleton_domain)

        if len(state_domain_dict[key]) <= 0:
            out += key.lower() + "," + "White" + "\n"
            failed = True
            failed_index = i
            break
        else:
            failed = False
        if key not in visited_node and len(state_domain_dict[key]) > 0:
            if is_backtrack and selected_dict.get(key, None) is not None:
                previous_selected_parent_domain = selected_dict.get(key).pop()
                if previous_selected_parent_domain in state_domain_dict[key]:
                    state_domain_dict[key].remove(previous_selected_parent_domain)
                if len(selected_dict[key]) == 0:
                    del selected_dict[key]
            if parent_domain_to_be_chosen is not None and key == state_list[0]:
                parent_domain = parent_domain_to_be_chosen
                previous_selected_parent_domain = selected_root_domain_values[-1]
            else:
                if heuristic_value == 3:
                    parent_domain = lcv(key, state_domain_dict, neighbor_state_dict, selected_dict)
                else:
                    parent_domain = state_domain_dict[key][0]
            state_domain_dict[key].remove(parent_domain)
            selected_dict.setdefault(key,list()).append(parent_domain)
            visited_node.add(key)
            #print('Selected Dct', selected_dict)
            for neighbours in neighbor_state_dict[key]:
                if previous_selected_parent_domain is not None and previous_selected_parent_domain not in state_domain_dict[neighbours]:
                    if neighbours in selected_dict.keys() and previous_selected_parent_domain not in selected_dict[neighbours]:
                        state_domain_dict[neighbours].append(previous_selected_parent_domain)
                if parent_domain in state_domain_dict[neighbours] and neighbours not in list(visited_node):
                    state_domain_dict[neighbours].remove(parent_domain)
        i += 1
        out += key.lower() + "," + parent_domain + "\n"
    if failed:
        length = len(list( selected_backtrack_node))
        if length > 0:
            last_element = list(sorted(selected_backtrack_node))[0]
            failed_index = last_element - 1
        else:
            failed_index -= 1
        while len(state_domain_dict[state_list[failed_index]]) <= 1:
            failed_index -= 1
        selected_backtrack_node.add(failed_index)
        if failed_index > 0:
            visited_node.clear()
            forward_checking_singleton(failed_index, state_domain_dict, neighbor_state_dict, True, None)
        else:
            state_domain_dict_list = ['Red', 'Green', 'Blue', 'Yellow']
            previous_root_values = selected_dict[state_list[0]]
            selected_root_domain_values.extend(previous_root_values)
            for previous_domain in selected_root_domain_values:
                    state_domain_dict_list.remove(previous_domain)
            if len(state_domain_dict_list) == 0:
                return
            forward_checking_singleton(0, domains, united_stated_of_america, True, state_domain_dict_list[0])
    else:
        pass

    
def singleton(state_domain_dict, neighbor_state_dict, key, visited_node):
    domain_list = []
    for neighbours in neighbor_state_dict[key]:
        if neighbours in visited_node:
            continue
        if len(state_domain_dict[neighbours]) == 1:
            domain_list.append(state_domain_dict[neighbours][0])
    return domain_list

In [14]:
R  = 'Red'
G  = 'Green'
B  = 'Blue'
Y  = 'Yellow'
AL = "Alabama"
AK = "Alaska"
AZ = "Arizona"
AR = "Arkansas"
CA = "California"
CO = "Colorado"
CT = "Connecticut"
DE = "Delaware"
FL = "Florida"
GA = "Georgia"
HI = "Hawaii"
ID = "Idaho"
IL = "Illinois"
IN = "Indiana"
IA = "Iowa"
KS = "Kansas"
KY = "Kentucky"
LA = "Louisiana"
ME = "Maine"
MD = "Maryland"
MA = "Massachusetts"
MI = "Michigan"
MN = "Minnesota"
MS = "Mississippi"
MO = "Missouri"
MT = "Montana"
NE = "Nebraska"
NV = "Nevada"
NH = "NewHampshire"
NJ = "NewJersey"
NM = "NewMexico"
NY = "NewYork"
NC = "NorthCarolina"
ND = "NorthDakota"
OH = "Ohio"
OK = "Oklahoma"
OR = "Oregon"
PA = "Pennsylvania"
RI = "RhodeIsland"
SC = "SouthCarolina"
SD = "SouthDakota"
TN = "Tennessee"
TX = "Texas"
UT = "Utah"
VT = "Vermont"
VA = "Virginia"
WA = "Washington"
WV = "WestVirginia"
WI = "Wisconsin"
WY = "Wyoming"


global domains 
domains= {
    AL: [R,G,B,Y],
    AK: [R,G,B,Y],
    AZ: [R,G,B,Y],
    AR: [R,G,B,Y],
    CA: [R,G,B,Y],
    CO: [R,G,B,Y],
    CT: [R,G,B,Y],
    DE: [R,G,B,Y],
    FL: [R,G,B,Y],
    GA: [R,G,B,Y],
    HI: [R,G,B,Y],
    ID: [R,G,B,Y],
    IL: [R,G,B,Y],
    IN: [R,G,B,Y],
    IA: [R,G,B,Y],
    KS: [R,G,B,Y],
    KY: [R,G,B,Y],
    LA: [R,G,B,Y],
    ME: [R,G,B,Y],
    MD: [R,G,B,Y],
    MA: [R,G,B,Y],
    MI: [R,G,B,Y],
    MN: [R,G,B,Y],
    MS: [R,G,B,Y],
    MO: [R,G,B,Y],
    MT: [R,G,B,Y],
    NE: [R,G,B,Y],
    NV: [R,G,B,Y],
    NH: [R,G,B,Y],
    NJ: [R,G,B,Y],
    NM: [R,G,B,Y],
    NY: [R,G,B,Y],
    NC: [R,G,B,Y],
    ND: [R,G,B,Y],
    OH: [R,G,B,Y],
    OK: [R,G,B,Y],
    OR: [R,G,B,Y],
    PA: [R,G,B,Y],
    RI: [R,G,B,Y],
    SC: [R,G,B,Y],
    SD: [R,G,B,Y],
    TN: [R,G,B,Y],
    TX: [R,G,B,Y],
    UT: [R,G,B,Y],
    VT: [R,G,B,Y],
    VA: [R,G,B,Y],
    WA: [R,G,B,Y],
    WV: [R,G,B,Y],
    WI: [R,G,B,Y],
    WY: [R,G,B,Y],
}

global united_stated_of_america
united_stated_of_america = {
    AL: {GA, FL, TN, MS},
    AK: {},
    AZ: {CA, NV, UT, CO, NM},
    AR: {MO, OK, TX, LA, TN, MS},
    CA: {OR, NV, AZ},
    CO: {WY, NE, KS, OK, NM, AZ, UT},
    CT: {NY, MA, RI},
    DE: {NJ, PA, MD},
    FL: {AL, GA},
    GA: {SC, NC, TN, AL, FL},
    HI: {},
    ID: {WA, MT, OR, WY, UT, NV},
    IL: {WI, IA, MO, KY, IN, MI},
    IN: {MI, WI, IL, KY, OH},
    IA: {MN, SD, NE, MO, WI, IL},
    KS: {NE, CO, OK, MO},
    KY: {IN, IL, MO, TN, OH, WV, VA},
    LA: {AR, TX, MS},
    ME: {NH},
    MD: {DE, PA, VA, WV, OH},
    MA: {NY, VT, NH, CT, RI},
    MI: {IL, WI, IN, OH},
    MN: {ND, SD, IA, WI},
    MS: {TN, AR, LA, AL},
    MO: {IA, NE, KS, OK, AR, IL, KY, TN},
    MT: {ID, WY, SD, ND},
    NE: {SD, WY, CO, KS, MO, IA},
    NV: {OR, ID, UT, AZ, CA},
    NH: {VT, ME, MA},
    NJ: {DE, PA, NY},
    NM: {AZ, UT, CO, OK, TX},
    NY: {PA, NJ, VT, MA, CT},
    NC: {GA, TN, SC, VA},
    ND: {MT, SD, MN},
    OH: {MI, IN, KY, WV, PA, MD},
    OK: {KS, CO, NM, TX, AR, MO},
    OR: {WA, ID, NV, CA},
    PA: {DE, OH, WV, MD, NJ, NY},
    RI: {MA, CT},
    SC: {GA, NC},
    SD: {ND, MT, WY, NE, MN, IA},
    TN: {KY, MO, AR, MS, MO, AL, GA, NC, VA},
    TX: {OK, NM, AR, LA},
    UT: {ID, NV, WY, CO, AZ, NM},
    VT: {NY, NH, MA},
    VA: {WV, KY, NC, MD, TN},
    WA: {OR, ID},
    WV: {OH, VA, KY, PA, MD},
    WI: {MN, IA, IL, MI, IN},
    WY: {MT, SD, NE, CO, UT, ID},
}
global state_list
state_list = [WA, OR, CA, NV, ID, MT, WY, UT, AZ, NM, CO, MO, AR, LA, MS, AL, FL, GA, SC, NC, TN, KY, VA, DE, MD, WV, OH, IN, IL, WI, MI, PA, NJ, CT, RI, NY, VT, NH, MA, ME, AK, HI]

# Backtrack + Forward Checking + Singleton - without heuristics

In [15]:
neighbor_state_dict = united_stated_of_america
state_domain_dict = domains

start_time = time.time()

#forward checking without heuristic with heuristic value 0
forward_checking_singleton(0, domains, united_stated_of_america, heuristic_value = 0)

print('\nTime taken:', time.time() - start_time)
print("\nBacktrack Count", back_track_count)
print('\nVisited Node', visited_node)
print('\n')
state_count = 0
for state in selected_dict:
    print('Assigned Color to state ', state, '->', selected_dict[state])
    state_count += 1

print('statecount:', state_count)


Time taken: 0.0004291534423828125

Backtrack Count 0

Visited Node {'Indiana', 'Illinois', 'NewMexico', 'Virginia', 'Tennessee', 'Nevada', 'SouthCarolina', 'Delaware', 'Wisconsin', 'Pennsylvania', 'NewYork', 'Vermont', 'Florida', 'Maine', 'Connecticut', 'Arkansas', 'California', 'WestVirginia', 'Alaska', 'Oregon', 'Mississippi', 'Kentucky', 'Missouri', 'NewJersey', 'Idaho', 'Arizona', 'Utah', 'RhodeIsland', 'Georgia', 'Maryland', 'Massachusetts', 'Washington', 'Michigan', 'Colorado', 'NorthCarolina', 'Hawaii', 'Wyoming', 'Ohio', 'NewHampshire', 'Louisiana', 'Alabama', 'Montana'}


Assigned Color to state  Washington -> ['Red']
Assigned Color to state  Oregon -> ['Green']
Assigned Color to state  California -> ['Red']
Assigned Color to state  Nevada -> ['Blue']
Assigned Color to state  Idaho -> ['Yellow']
Assigned Color to state  Montana -> ['Red']
Assigned Color to state  Wyoming -> ['Green']
Assigned Color to state  Utah -> ['Red']
Assigned Color to state  Arizona -> ['Green']
Assign

# Backtrack + Forward Checking + Singleton - with MRV heuristics

In [13]:
neighbor_state_dict = united_stated_of_america
state_domain_dict = domains
start_time = time.time()
#forward checking without heuristic with heuristic value 0
forward_checking_singleton(0, domains, united_stated_of_america, heuristic_value = 1)

print('\nTime taken:', time.time() - start_time)
print("\nBacktrack Count", back_track_count)
print('\nVisited Node', visited_node)
print('\n')
state_count = 0
for state in selected_dict:
    print('Assigned Color to state ', state, '->', selected_dict[state])
    state_count += 1

print('statecount:', state_count)


Time taken: 0.0015187263488769531

Backtrack Count 0

Visited Node {'Iowa', 'Wisconsin', 'Illinois', 'Texas', 'Maryland', 'Louisiana', 'Michigan', 'California', 'Oklahoma', 'Kentucky', 'WestVirginia', 'Utah', 'NewYork', 'NewJersey', 'Alabama', 'Georgia', 'NorthDakota', 'Pennsylvania', 'Ohio', 'Arkansas', 'Colorado', 'Minnesota', 'Idaho', 'Nebraska', 'Virginia', 'Missouri', 'Arizona', 'NewMexico', 'Oregon', 'Kansas', 'Indiana', 'Florida', 'Tennessee', 'SouthCarolina', 'Delaware', 'Washington', 'Mississippi', 'SouthDakota', 'Montana', 'Wyoming', 'NorthCarolina', 'Nevada'}


Assigned Color to state  Missouri -> ['Red']
Assigned Color to state  Tennessee -> ['Green']
Assigned Color to state  Kentucky -> ['Blue']
Assigned Color to state  Arkansas -> ['Blue']
Assigned Color to state  Illinois -> ['Green']
Assigned Color to state  Iowa -> ['Blue']
Assigned Color to state  Nebraska -> ['Green']
Assigned Color to state  Oklahoma -> ['Green']
Assigned Color to state  SouthDakota -> ['Red']
Assi

# Backtrack + Forward Checking + Singleton - with Degree heuristics

In [ ]:
neighbor_state_dict = united_stated_of_america
state_domain_dict = domains
start_time = time.time()
#forward checking without heuristic with heuristic value 0
forward_checking_singleton(0, domains, united_stated_of_america, heuristic_value = 2)

print('\nTime taken:', time.time() - start_time)
print("\nBacktrack Count", back_track_count)
print('\nVisited Node', visited_node)
print('\n')
state_count = 0
for state in selected_dict:
    print('Assigned Color to state ', state, '->', selected_dict[state])
    state_count += 1

print('statecount:', state_count)

# Backtrack + Forward Checking + Singleton - with LCS heuristics

In [131]:
neighbor_state_dict = united_stated_of_america
state_domain_dict = domains
start_time = time.time()
#forward checking without heuristic with heuristic value 0
forward_checking_singleton(0, domains, united_stated_of_america, heuristic_value = 3)

print('\nTime taken:', time.time() - start_time)
print("\nBacktrack Count", back_track_count)
print('\nVisited Node', visited_node)
print('\n')
state_count = 0
for state in selected_dict:
    print('Assigned Color to state ', state, '->', selected_dict[state])
    state_count += 1

print('statecount:', state_count)


Time taken: 0.0008182525634765625

Backtrack Count 0

Visited Node {'Mississippi', 'Texas', 'Ohio', 'Alabama', 'Oklahoma', 'Maryland', 'Idaho', 'Nebraska', 'NewYork', 'Georgia', 'Pennsylvania', 'RhodeIsland', 'Arkansas', 'NewHampshire', 'Indiana', 'Nevada', 'Kansas', 'Missouri', 'Iowa', 'NorthCarolina', 'Illinois', 'NewJersey', 'Arizona', 'NorthDakota', 'Maine', 'Louisiana', 'SouthCarolina', 'Kentucky', 'Massachusetts', 'Minnesota', 'Delaware', 'California', 'Florida', 'Tennessee', 'Virginia', 'Oregon', 'Montana', 'Michigan', 'Colorado', 'Hawaii', 'Wisconsin', 'Vermont', 'Utah', 'SouthDakota', 'Washington', 'Alaska', 'Connecticut', 'WestVirginia', 'Wyoming', 'NewMexico'}


Assigned Color to state  Washington -> ['Red']
Assigned Color to state  Oregon -> ['Green']
Assigned Color to state  California -> ['Red']
Assigned Color to state  Nevada -> ['Blue']
Assigned Color to state  Idaho -> ['Yellow']
Assigned Color to state  Montana -> ['Red']
Assigned Color to state  Wyoming -> ['Green']